In [5]:
import os
import shutil
import glob
import logging
import pathlib
from pathlib import Path
from tempfile import gettempdir, NamedTemporaryFile
from typing import List, Dict, Union

import subprocess

import pandas as pd
import numpy as np

import datamol as dm
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools

from openmm.app import PDBFile
from pdbfixer import PDBFixer
from pymol import cmd

from openeye import oechem
from openeye import oequacpac

In [6]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.StreamHandler()
                    ])
logger = logging.getLogger(__name__)

In [7]:
from posebusters import PoseBusters

from Pymol_Docking import Plants_Docking
from Pymol_Docking import Pymol_Docking

In [4]:
pymol = Pymol_Docking("./LAC3.pdb", "CC1=C(NC2CCCC2)N2C=CC=CC2=N1")
docking_result = pymol.run_smina_docking("Dock", "XXX")

2025-02-11 19:56:22,905 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): proteins.plus:443


Running protein preparation
Preparing protein with ProtoSS ...


2025-02-11 19:56:23,126 - urllib3.connectionpool - DEBUG - https://proteins.plus:443 "POST /api/v2/protoss/ HTTP/1.1" 202 77
2025-02-11 19:56:23,131 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): proteins.plus:443
2025-02-11 19:56:23,281 - urllib3.connectionpool - DEBUG - https://proteins.plus:443 "GET /api/v2/protoss/jobs/4f2cd205-31a2-4baf-af14-acba0bbfef76/ HTTP/1.1" 200 270
2025-02-11 19:56:23,284 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): proteins.plus:443
2025-02-11 19:56:23,408 - urllib3.connectionpool - DEBUG - https://proteins.plus:443 "GET /api/v2/molecule_handler/proteins/0314ca22-5bc1-48ef-89e0-bfde6c777b47/ HTTP/1.1" 200 282604


Job 4f2cd205-31a2-4baf-af14-acba0bbfef76 completed with success


2025-02-11 19:56:23,562 - fsspec.local - DEBUG - open file: /tmp/TMP_SMILES.sdf


Running docking


In [5]:
import rdkit.Chem as rdChem

In [6]:
def pose_buster_processer(mol_pred: Path, mol_crystal: Path, mol_prot: Path):
    from posebusters import PoseBusters
    buster = PoseBusters()
    df = buster.bust(mol_pred, mol_crystal, mol_prot)
    df.reset_index(drop=False, inplace=True)
    bool_columns = df.select_dtypes(include=[bool]).columns
    success_rate_series = df[bool_columns].mean(axis=1) * 100
    success_rate_list = success_rate_series.tolist()
    
    # Read all molecules first
    mol_list = list(rdChem.SDMolSupplier(mol_pred.as_posix()))
    
    # Create a temporary file path
    temp_path = mol_pred.parent / f"temp_{mol_pred.name}"
    
    # Write to temporary file
    with rdChem.SDWriter(temp_path.as_posix()) as writer:
        for pose, success_rate in zip(mol_list, success_rate_list):
            pose.SetProp("Compliance", str(success_rate))  # Convert to string
            writer.write(pose)
    
    # Replace original file with temporary file
    temp_path.replace(mol_pred)
    
    return mol_pred

In [ ]:
pose_buster_processer(docking_result, pymol.crystal_sdf, pymol.protein_preared)

In [ ]:
buster = PoseBusters()

In [ ]:
df = buster.bust(docking_result, pymol.crystal_sdf, pymol.protein_preared).reset_index(drop=False)

In [18]:
bool_columns = df.select_dtypes(include=[bool]).columns
X = df[bool_columns].mean(axis=1) * 100

In [ ]:
success_rate_list = X.tolist()

In [ ]:
df = buster.bust("./XXX.sdf", "./Crystal.sdf", "./LAC3.pdb")

In [30]:
df.reset_index(drop=False, inplace=True)

In [32]:
bool_columns = df.select_dtypes(include=[bool]).columns

In [34]:
df['success_rate'] = df[bool_columns].mean(axis=1) * 100

In [ ]:
df

In [ ]:
plants = Plants_Docking(Path("./LAC3.pdb"), "c1ccccc1")
plants.run_plants_docking(10)